In [37]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows',98)
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [29]:
df=pd.read_csv("./data/pre_labeled_weather.csv")
df=df.iloc[:,1:]
df

,location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindGustDir,Sunshine,Evaporation,TodayRain,TomorrowRain,...,Temp9am,Pressure3pm,WindDir3pm,Cloud3pm,Humidity3pm,WindSpeed3pm,Temp3pm,Year,Month,Day
0,45,12.6,23.2,0.5,6.4,7,13.17,3.2,0,0,...,18.3,998.7,9,6.0,51.0,4.2,19.3,2012,5,5
1,45,10.6,24.1,1.8,6.5,15,13.17,3.2,0,0,...,18.6,1000.8,0,2.0,35.0,2.7,20.5,2012,5,6
2,45,14.2,25.1,1.8,7.0,7,13.17,3.2,0,0,...,22.2,1002.8,0,0.0,31.0,2.1,22.9,2012,5,7
3,45,12.1,21.1,1.8,6.1,7,13.17,3.2,0,0,...,17.0,1005.5,2,3.0,70.0,2.8,18.3,2012,5,8
4,45,11.9,15.4,1.8,6.5,7,13.17,3.2,0,1,...,14.7,1013.2,6,9.0,89.0,4.6,13.9,2012,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343701,17,9.8,20.3,1.8,4.8,14,13.17,3.2,0,0,...,14.4,1006.8,8,1.0,35.0,2.4,19.5,2022,5,1
343702,17,7.9,21.5,1.8,5.4,14,13.17,3.2,0,0,...,15.9,1006.5,15,7.0,42.0,2.2,20.6,2022,5,2
343703,17,8.3,19.9,1.8,3.6,11,13.17,3.2,0,0,...,13.0,1014.3,8,0.0,37.0,2.7,18.9,2022,5,3
343704,17,8.2,22.7,1.8,4.1,11,13.17,3.2,0,0,...,15.5,1014.0,11,0.0,39.0,2.6,21.3,2022,5,4


In [30]:
y_label = df.TomorrowRain
X_features = df.drop("TomorrowRain",axis=1)

In [31]:
X_train, X_test, y_train, y_test= train_test_split(X_features, y_label, test_size=0.2, random_state=180)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=180)
dtrain = xgb.DMatrix(data=X_train , label=y_train)
deval =xgb.DMatrix(data=X_val,label=y_val)
dtest = xgb.DMatrix(data=X_test , label=y_test)
print(X_train.shape ,X_val, X_test.shape)

(219971, 24)         location  MinTemp  MaxTemp  Rainfall  WindGustSpeed  WindGustDir  \
36623         23     -1.9      8.7       1.8            2.9           12   
238761        53     25.9     33.1       0.1            4.5           15   
89703         91     19.2     24.2      28.5            3.4            3   
6307          82     -3.4      4.1       0.0            3.8           11   
239157        53     17.2     31.9       1.8            5.0            5   
...          ...      ...      ...       ...            ...          ...   
137818         7      8.8     17.6       7.5            6.1            5   
282096        92     13.6     25.7       0.4            6.0           12   
337100         3     11.1     17.4      11.0            4.5            9   
159061        43      3.4     15.1       6.4            9.5           14   
323388         4     23.3     31.9       1.4            3.6            9   

        Sunshine  Evaporation  TodayRain  Pressure9am  ...  Temp9am  \
366

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [39]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = xgb.XGBClassifier(n_estimators=100)

params = {'max_depth':[5, 7] ,'eta': [0.1,0.3,0.5],'num_boost_rounds':[400,100,200] }

# cv는 3으로 지정 
gridcv = GridSearchCV(xgb_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="error",
           eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:',gridcv.best_params_) 

xgb_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

C:\Users\maini\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\maini\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[19:49:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_boost_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-error:0.19591	validation_1-error:0.19624
[1]	validation_0-error:0.19513	validation_1-error:0.19467
[2]	validation_0-error:0.19368	validation_1-error:0.19323
[3]	validation_0-error:0.19328	validation_1-error:0.19362
[4]	validation_0-error:0.19237	validation_1-error:0.19297
[5]	validation_0-error:0.19253	validation_1-error:0.19285
[6]	validation_0-error:0.19231	validation_1-error:0.19307
[7]	validation_0-error:0.19205	validation_1-error:0.19250
[8]	validation_0-error:0.19158	validation_1-error:0.19231
[9]	validation_0-error:0.19128	validation_1-error:0.1

[24]	validation_0-error:0.18767	validation_1-error:0.18844
[25]	validation_0-error:0.18759	validation_1-error:0.18815
[26]	validation_0-error:0.18735	validation_1-error:0.18821
[27]	validation_0-error:0.18704	validation_1-error:0.18796
[28]	validation_0-error:0.18682	validation_1-error:0.18776
[29]	validation_0-error:0.18654	validation_1-error:0.18722
[30]	validation_0-error:0.18652	validation_1-error:0.18705
[31]	validation_0-error:0.18633	validation_1-error:0.18684
[32]	validation_0-error:0.18617	validation_1-error:0.18676
[33]	validation_0-error:0.18613	validation_1-error:0.18663
[34]	validation_0-error:0.18593	validation_1-error:0.18658
[35]	validation_0-error:0.18579	validation_1-error:0.18638
[36]	validation_0-error:0.18539	validation_1-error:0.18625
[37]	validation_0-error:0.18526	validation_1-error:0.18636
[38]	validation_0-error:0.18503	validation_1-error:0.18620
[39]	validation_0-error:0.18481	validation_1-error:0.18606
[40]	validation_0-error:0.18468	validation_1-error:0.185

[55]	validation_0-error:0.18275	validation_1-error:0.18437
[56]	validation_0-error:0.18267	validation_1-error:0.18428
[57]	validation_0-error:0.18260	validation_1-error:0.18411
[58]	validation_0-error:0.18238	validation_1-error:0.18398
[59]	validation_0-error:0.18202	validation_1-error:0.18366
[60]	validation_0-error:0.18196	validation_1-error:0.18380
[61]	validation_0-error:0.18176	validation_1-error:0.18379
[62]	validation_0-error:0.18166	validation_1-error:0.18377
[63]	validation_0-error:0.18154	validation_1-error:0.18388
[64]	validation_0-error:0.18139	validation_1-error:0.18347
[65]	validation_0-error:0.18117	validation_1-error:0.18344
[66]	validation_0-error:0.18094	validation_1-error:0.18303
[67]	validation_0-error:0.18083	validation_1-error:0.18299
[68]	validation_0-error:0.18070	validation_1-error:0.18280
[69]	validation_0-error:0.18067	validation_1-error:0.18268
[70]	validation_0-error:0.18051	validation_1-error:0.18251
[71]	validation_0-error:0.18031	validation_1-error:0.182

[87]	validation_0-error:0.17914	validation_1-error:0.18127
[88]	validation_0-error:0.17898	validation_1-error:0.18117
[89]	validation_0-error:0.17893	validation_1-error:0.18116
[90]	validation_0-error:0.17880	validation_1-error:0.18105
[91]	validation_0-error:0.17874	validation_1-error:0.18111
[92]	validation_0-error:0.17865	validation_1-error:0.18098
[93]	validation_0-error:0.17864	validation_1-error:0.18100
[94]	validation_0-error:0.17860	validation_1-error:0.18110
[95]	validation_0-error:0.17852	validation_1-error:0.18088
[96]	validation_0-error:0.17828	validation_1-error:0.18086
[97]	validation_0-error:0.17806	validation_1-error:0.18078
[98]	validation_0-error:0.17783	validation_1-error:0.18069
[99]	validation_0-error:0.17785	validation_1-error:0.18041
[19:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_boost_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language

...

[33]	validation_0-error:0.15120	validation_1-error:0.17091
[34]	validation_0-error:0.15067	validation_1-error:0.17086
[35]	validation_0-error:0.15047	validation_1-error:0.17091
[36]	validation_0-error:0.15001	validation_1-error:0.17075
[37]	validation_0-error:0.14879	validation_1-error:0.17003
[38]	validation_0-error:0.14856	validation_1-error:0.17006
[39]	validation_0-error:0.14790	validation_1-error:0.16982
[40]	validation_0-error:0.14681	validation_1-error:0.16950
[41]	validation_0-error:0.14605	validation_1-error:0.16849
[42]	validation_0-error:0.14538	validation_1-error:0.16809
[43]	validation_0-error:0.14500	validation_1-error:0.16799
[44]	validation_0-error:0.14400	validation_1-error:0.16757
[45]	validation_0-error:0.14381	validation_1-error:0.16742
[46]	validation_0-error:0.14333	validation_1-error:0.16707
[47]	validation_0-error:0.14251	validation_1-error:0.16671
[48]	validation_0-error:0.14183	validation_1-error:0.16718
[49]	validation_0-error:0.14151	validation_1-error:0.166

[65]	validation_0-error:0.12700	validation_1-error:0.15538
[66]	validation_0-error:0.12665	validation_1-error:0.15499
[67]	validation_0-error:0.12643	validation_1-error:0.15499
[68]	validation_0-error:0.12560	validation_1-error:0.15487
[69]	validation_0-error:0.12543	validation_1-error:0.15491
[70]	validation_0-error:0.12538	validation_1-error:0.15497
[71]	validation_0-error:0.12518	validation_1-error:0.15515
[72]	validation_0-error:0.12457	validation_1-error:0.15496
[73]	validation_0-error:0.12378	validation_1-error:0.15512
[74]	validation_0-error:0.12347	validation_1-error:0.15535
[75]	validation_0-error:0.12327	validation_1-error:0.15563
[76]	validation_0-error:0.12251	validation_1-error:0.15541
[77]	validation_0-error:0.12223	validation_1-error:0.15560
[78]	validation_0-error:0.12205	validation_1-error:0.15548
[79]	validation_0-error:0.12206	validation_1-error:0.15565
[80]	validation_0-error:0.12166	validation_1-error:0.15547
[81]	validation_0-error:0.12155	validation_1-error:0.155

[99]	validation_0-error:0.11528	validation_1-error:0.15810
[19:53:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_boost_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-error:0.19005	validation_1-error:0.19262
[1]	validation_0-error:0.18558	validation_1-error:0.18959
[2]	validation_0-error:0.18454	validation_1-error:0.18856
[3]	validation_0-error:0.18220	validation_1-error:0.18692
[4]	validation_0-error:0.18149	validation_1-error:0.18683
[5]	validation_0-error:0.18026	validation_1-error:0.18539
[6]	validation_0-error:0.17926	validation_1-error:0.18417
[7]	validation_0-error:0.17799	validation_1-error:0.18414
[8]	validation_0-error:0.17623	validation_1-error:0.

[23]	validation_0-error:0.15568	validation_1-error:0.16723
[24]	validation_0-error:0.15522	validation_1-error:0.16751
[25]	validation_0-error:0.15456	validation_1-error:0.16722
[26]	validation_0-error:0.15303	validation_1-error:0.16704
[27]	validation_0-error:0.15247	validation_1-error:0.16693
[28]	validation_0-error:0.15206	validation_1-error:0.16688
[29]	validation_0-error:0.15198	validation_1-error:0.16668
[30]	validation_0-error:0.15135	validation_1-error:0.16597
[31]	validation_0-error:0.15057	validation_1-error:0.16517
[32]	validation_0-error:0.15028	validation_1-error:0.16511
[33]	validation_0-error:0.14999	validation_1-error:0.16588
[34]	validation_0-error:0.14937	validation_1-error:0.16566
[35]	validation_0-error:0.14899	validation_1-error:0.16556
[36]	validation_0-error:0.14826	validation_1-error:0.16520
[37]	validation_0-error:0.14805	validation_1-error:0.16528
[38]	validation_0-error:0.14719	validation_1-error:0.16533
[39]	validation_0-error:0.14504	validation_1-error:0.163

[56]	validation_0-error:0.12755	validation_1-error:0.15823
[57]	validation_0-error:0.12709	validation_1-error:0.15784
[58]	validation_0-error:0.12672	validation_1-error:0.15779
[59]	validation_0-error:0.12578	validation_1-error:0.15740
[60]	validation_0-error:0.12511	validation_1-error:0.15704
[61]	validation_0-error:0.12446	validation_1-error:0.15677
[62]	validation_0-error:0.12407	validation_1-error:0.15695
[63]	validation_0-error:0.12370	validation_1-error:0.15666
[64]	validation_0-error:0.12300	validation_1-error:0.15660
[65]	validation_0-error:0.12220	validation_1-error:0.15573
[66]	validation_0-error:0.12109	validation_1-error:0.15468
[67]	validation_0-error:0.12071	validation_1-error:0.15469
[68]	validation_0-error:0.12000	validation_1-error:0.15426
[69]	validation_0-error:0.11833	validation_1-error:0.15301
[70]	validation_0-error:0.11725	validation_1-error:0.15231
[71]	validation_0-error:0.11715	validation_1-error:0.15229
[72]	validation_0-error:0.11664	validation_1-error:0.152

In [40]:
params = { 'max_depth':9,
           'eta': 0.5,
           'objective':'binary:logistic',
           'eval_metric':'logloss'
        }
num_rounds = 400

In [41]:
params = { 'max_depth':9,
           'eta': 0.5,
           'objective':'binary:logistic',
           'eval_metric':'logloss'
        }
num_rounds = 400
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(deval,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , \
                      early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])
get_clf_eval(y_test , preds, pred_probs)

[0]	train-logloss:0.52736	eval-logloss:0.52814
[1]	train-logloss:0.47037	eval-logloss:0.47289
[2]	train-logloss:0.44545	eval-logloss:0.44888
[3]	train-logloss:0.43098	eval-logloss:0.43651
[4]	train-logloss:0.42314	eval-logloss:0.42996
[5]	train-logloss:0.41755	eval-logloss:0.42585
[6]	train-logloss:0.41285	eval-logloss:0.42252
[7]	train-logloss:0.40490	eval-logloss:0.41475
[8]	train-logloss:0.40137	eval-logloss:0.41278
[9]	train-logloss:0.39865	eval-logloss:0.41195
[10]	train-logloss:0.39546	eval-logloss:0.41026
[11]	train-logloss:0.38936	eval-logloss:0.40495
[12]	train-logloss:0.38324	eval-logloss:0.40010
[13]	train-logloss:0.37930	eval-logloss:0.39734
[14]	train-logloss:0.37595	eval-logloss:0.39502
[15]	train-logloss:0.37395	eval-logloss:0.39454
[16]	train-logloss:0.37120	eval-logloss:0.39343
[17]	train-logloss:0.36842	eval-logloss:0.39219
[18]	train-logloss:0.36631	eval-logloss:0.39151
[19]	train-logloss:0.36180	eval-logloss:0.38739
[20]	train-logloss:0.35965	eval-logloss:0.38641
[2

[170]	train-logloss:0.20629	eval-logloss:0.33053
[171]	train-logloss:0.20605	eval-logloss:0.33061
[172]	train-logloss:0.20558	eval-logloss:0.33068
[173]	train-logloss:0.20522	eval-logloss:0.33049
[174]	train-logloss:0.20519	eval-logloss:0.33052
[175]	train-logloss:0.20512	eval-logloss:0.33058
[176]	train-logloss:0.20505	eval-logloss:0.33059
[177]	train-logloss:0.20468	eval-logloss:0.33060
[178]	train-logloss:0.20438	eval-logloss:0.33059
[179]	train-logloss:0.20415	eval-logloss:0.33053
[180]	train-logloss:0.20364	eval-logloss:0.33055
[181]	train-logloss:0.20304	eval-logloss:0.33070
[182]	train-logloss:0.20243	eval-logloss:0.33068
[183]	train-logloss:0.20157	eval-logloss:0.33035
[184]	train-logloss:0.20097	eval-logloss:0.33028
[185]	train-logloss:0.20071	eval-logloss:0.33039
[186]	train-logloss:0.20007	eval-logloss:0.33040
[187]	train-logloss:0.19987	eval-logloss:0.33044
[188]	train-logloss:0.19960	eval-logloss:0.33048
[189]	train-logloss:0.19936	eval-logloss:0.33053
[190]	train-logloss:

In [43]:
params = { 'max_depth':9,
           'eta': 0.7,
           'objective':'binary:logistic',
           'eval_metric':'logloss'
        }
num_rounds = 400
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(deval,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , \
                      early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])
get_clf_eval(y_test , preds, pred_probs)

[0]	train-logloss:0.47854	eval-logloss:0.48459
[1]	train-logloss:0.42811	eval-logloss:0.44133
[2]	train-logloss:0.40558	eval-logloss:0.42570
[3]	train-logloss:0.39210	eval-logloss:0.41711
[4]	train-logloss:0.37759	eval-logloss:0.40625
[5]	train-logloss:0.36745	eval-logloss:0.40187
[6]	train-logloss:0.36185	eval-logloss:0.40120
[7]	train-logloss:0.34952	eval-logloss:0.39359
[8]	train-logloss:0.34282	eval-logloss:0.38932
[9]	train-logloss:0.32931	eval-logloss:0.38031
[10]	train-logloss:0.32073	eval-logloss:0.37829
[11]	train-logloss:0.31557	eval-logloss:0.37691
[12]	train-logloss:0.31213	eval-logloss:0.37696
[13]	train-logloss:0.31045	eval-logloss:0.37687
[14]	train-logloss:0.30583	eval-logloss:0.37696
[15]	train-logloss:0.29795	eval-logloss:0.37124
[16]	train-logloss:0.29097	eval-logloss:0.36856
[17]	train-logloss:0.28692	eval-logloss:0.36872
[18]	train-logloss:0.28456	eval-logloss:0.36887
[19]	train-logloss:0.28268	eval-logloss:0.36955
[20]	train-logloss:0.27649	eval-logloss:0.36838
[2

In [19]:
params = { 'max_depth':11,
           'eta': 0.7,
           'objective':'binary:logistic',
           'eval_metric':'logloss'
        }
num_rounds = 400
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(deval,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , \
                      early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])
get_clf_eval(y_test , preds, pred_probs)

[0]	train-aucpr:0.59497	eval-aucpr:0.47127
[1]	train-aucpr:0.62217	eval-aucpr:0.49404
[2]	train-aucpr:0.64499	eval-aucpr:0.51154
[3]	train-aucpr:0.65896	eval-aucpr:0.52145
[4]	train-aucpr:0.67084	eval-aucpr:0.52977
[5]	train-aucpr:0.68444	eval-aucpr:0.53931
[6]	train-aucpr:0.69673	eval-aucpr:0.54712
[7]	train-aucpr:0.70629	eval-aucpr:0.55466
[8]	train-aucpr:0.71290	eval-aucpr:0.55914
[9]	train-aucpr:0.72269	eval-aucpr:0.56530
[10]	train-aucpr:0.73002	eval-aucpr:0.56847
[11]	train-aucpr:0.73695	eval-aucpr:0.57329
[12]	train-aucpr:0.74495	eval-aucpr:0.58041
[13]	train-aucpr:0.74967	eval-aucpr:0.58239
[14]	train-aucpr:0.75469	eval-aucpr:0.58537
[15]	train-aucpr:0.76145	eval-aucpr:0.59084
[16]	train-aucpr:0.76788	eval-aucpr:0.59509
[17]	train-aucpr:0.77588	eval-aucpr:0.60092
[18]	train-aucpr:0.78016	eval-aucpr:0.60343
[19]	train-aucpr:0.78758	eval-aucpr:0.60806
[20]	train-aucpr:0.79347	eval-aucpr:0.61145
[21]	train-aucpr:0.80151	eval-aucpr:0.61756
[22]	train-aucpr:0.80692	eval-aucpr:0.6209

In [20]:
params = { 'max_depth':8,
           'eta': 0.2,
           'objective':'binary:logistic',
           'eval_metric':'aucpr'
        }
num_rounds = 600
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(dtest,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , \
                      early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])
get_clf_eval(y_test , preds, pred_probs)

[0]	train-aucpr:0.59497	eval-aucpr:0.47127
[1]	train-aucpr:0.62217	eval-aucpr:0.49404
[2]	train-aucpr:0.64499	eval-aucpr:0.51154
[3]	train-aucpr:0.65896	eval-aucpr:0.52145
[4]	train-aucpr:0.67084	eval-aucpr:0.52977
[5]	train-aucpr:0.68444	eval-aucpr:0.53931
[6]	train-aucpr:0.69673	eval-aucpr:0.54712
[7]	train-aucpr:0.70629	eval-aucpr:0.55466
[8]	train-aucpr:0.71290	eval-aucpr:0.55914
[9]	train-aucpr:0.72269	eval-aucpr:0.56530
[10]	train-aucpr:0.73002	eval-aucpr:0.56847
[11]	train-aucpr:0.73695	eval-aucpr:0.57329
[12]	train-aucpr:0.74495	eval-aucpr:0.58041
[13]	train-aucpr:0.74967	eval-aucpr:0.58239
[14]	train-aucpr:0.75469	eval-aucpr:0.58537
[15]	train-aucpr:0.76145	eval-aucpr:0.59084
[16]	train-aucpr:0.76788	eval-aucpr:0.59509
[17]	train-aucpr:0.77588	eval-aucpr:0.60092
[18]	train-aucpr:0.78016	eval-aucpr:0.60343
[19]	train-aucpr:0.78758	eval-aucpr:0.60806
[20]	train-aucpr:0.79347	eval-aucpr:0.61145
[21]	train-aucpr:0.80151	eval-aucpr:0.61756
[22]	train-aucpr:0.80692	eval-aucpr:0.6209

[185]	train-aucpr:0.98819	eval-aucpr:0.75244
[186]	train-aucpr:0.98822	eval-aucpr:0.75242
[187]	train-aucpr:0.98826	eval-aucpr:0.75242
[188]	train-aucpr:0.98834	eval-aucpr:0.75243
[189]	train-aucpr:0.98850	eval-aucpr:0.75251
[190]	train-aucpr:0.98894	eval-aucpr:0.75303
[191]	train-aucpr:0.98920	eval-aucpr:0.75365
[192]	train-aucpr:0.98930	eval-aucpr:0.75393
[193]	train-aucpr:0.98941	eval-aucpr:0.75386
[194]	train-aucpr:0.98957	eval-aucpr:0.75392
[195]	train-aucpr:0.98968	eval-aucpr:0.75474
[196]	train-aucpr:0.98984	eval-aucpr:0.75498
[197]	train-aucpr:0.98995	eval-aucpr:0.75488
[198]	train-aucpr:0.99006	eval-aucpr:0.75496
[199]	train-aucpr:0.99016	eval-aucpr:0.75486
predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.108 0.018 0.033 0.032 0.228 0.622 0.362 0.035 0.048 0.394]
예측값 10개만 표시: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
오차 행렬
[[53322  1498]
 [ 7264  6658]]
정확도: 0.8725, 정밀도: 0.8163, 재현율: 0.4782,    F1: 0.6031, AUC:0.9083


In [ ]:
params = { 'max_depth':8,
           'eta': 0.2,
           'objective':'binary:logistic',
           'eval_metric':'aucpr'
        }
num_rounds = 600
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(dtest,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , \
                      early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])
get_clf_eval(y_test , preds, pred_probs)